In [80]:
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple
from dataclasses import dataclass
from datetime import time

In [176]:
class Stop:
    def __init__(self, name: str, latitude: float, longitude: float) -> None:
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
    def __str__(self) -> str:
        return self.name + "[" + str(self.latitude) + ", " + str(self.longitude) + "]"
    def __repr__(self) -> str:
        return self.name + ", " + str(self.latitude) + "," + str(self.longitude)
    def __hash__(self) -> int:
        return hash(str(self.name))
    def __eq__(self, __value: object) -> bool:
        return isinstance(__value, Stop) and str(self.name) == str(__value.name)
    
@dataclass
class Route:
    line: str
    departure_minutes: int
    arrival_minutes: int

@dataclass
class StopRecord():
    min_arrival_minutes: int
    last_stop: Stop
    last_route: Route

In [186]:
real_timetable = pd.read_csv("connection_graph (1).csv")
real_timetable.set_index(keys='Unnamed: 0', inplace=True)
real_timetable.drop_duplicates(ignore_index=True, inplace=True)
len(real_timetable)

C:\Users\48791\AppData\Local\Temp\ipykernel_14628\1928706170.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  real_timetable = pd.read_csv("connection_graph (1).csv")


456019

In [187]:
def time_to_minutes(time_str: str) -> int:
    time_arr = time_str.split(':')
    hour=int(time_arr[0])
    minute=int(time_arr[1])
    return 60 * (hour % 24) + minute
def format_time(abnormal_time: int) -> str:
    return str(abnormal_time // 60).zfill(2) + ":" + str(abnormal_time % 60).zfill(2)

In [179]:
test_timetable = pd.read_csv("test_graph.csv")

In [189]:
timetable_dict: Dict[Stop, Dict[Stop, List[Route]]] = {}
for stop in real_timetable.itertuples():
    start_stop: Stop = Stop(stop.start_stop, stop.start_stop_lat, stop.start_stop_lon) 
    end_stop: Stop = Stop(stop.end_stop, stop.end_stop_lat, stop.end_stop_lon)
    route: Route = Route(line=stop.line, departure_minutes=time_to_minutes(stop.departure_time), arrival_minutes=time_to_minutes(stop.arrival_time))
    if start_stop not in timetable_dict.keys():
        timetable_dict[start_stop] = {end_stop: [route]}
    elif end_stop in timetable_dict[start_stop].keys():
        timetable_dict[start_stop][end_stop].append(route)
    else:
        timetable_dict[start_stop][end_stop] = [route] 
    if end_stop not in timetable_dict.keys():
        timetable_dict[end_stop] = {}
for stop, neighbors in timetable_dict.items():
    for _, routes in neighbors.items():
        routes.sort(key=lambda rt: rt.arrival_minutes)

In [223]:
start = Stop("Tramwajowa", 51.10446678,17.08466997)
end = Stop("Muchobór Wielki", 51.09892535,16.94155277)
# end = Stop("DWORZEC NADODRZE", 51.12431442,17.03503321)
start_time = '23:53:00'
change_minutes = 1
time = time_to_minutes(start_time)
stops_records: Dict[Stop, StopRecord] = {} 
for key in timetable_dict.keys():
    stops_records[key] = StopRecord(1e10, None, None)
stops_records[start] = StopRecord(time, None, None)
unseen_stops = list(timetable_dict.keys())
while(len(unseen_stops) > 0):
    curr_stop = unseen_stops[0]
    for stop in unseen_stops:
        if stops_records[stop].min_arrival_minutes < stops_records[curr_stop].min_arrival_minutes:
            curr_stop = stop
    unseen_stops.remove(curr_stop)
    arrival_minutes_modulo = stops_records[curr_stop].min_arrival_minutes % (24*60) 
    for neighbor, routes in timetable_dict[curr_stop].items():
        min_arrival_id = -1
        for i in range(len(routes)):
            change_fine = 0 if stops_records[curr_stop].last_route is not None and routes[i].line == stops_records[curr_stop].last_route.line else change_minutes
            if arrival_minutes_modulo + change_fine <= routes[i].departure_minutes:
                min_arrival_id = i
                break
        min_arrival_minutes = routes[min_arrival_id].arrival_minutes if min_arrival_id > -1 else 24*60+routes[0].arrival_minutes
        min_arrival_id = max(min_arrival_id,0)
        min_arrival_minutes += (stops_records[curr_stop].min_arrival_minutes // (24*60)) * 24*60 
        min_arrival_minutes += 0 if routes[min_arrival_id].arrival_minutes > routes[min_arrival_id].departure_minutes else 24*60
        if min_arrival_minutes < stops_records[neighbor].min_arrival_minutes:
            stops_records[neighbor].min_arrival_minutes = min_arrival_minutes
            stops_records[neighbor].last_stop = curr_stop
            stops_records[neighbor].last_route = routes[min_arrival_id]
temp = end, stops_records[end]
route: List[Tuple[Stop, StopRecord]] = []
max_length = [0,0,0,0,0]
while temp[1].last_stop is not None:
    route.append(temp)
    for i, element in enumerate([temp[1].last_route.line, temp[1].last_stop.name, format_time(temp[1].last_route.departure_minutes), temp[0].name, format_time(temp[1].last_route.arrival_minutes)]):
        max_length[i] = len(str(element)) if len(str(element)) > max_length[i] else max_length[i]
    temp = temp[1].last_stop, stops_records[temp[1].last_stop]
route.reverse()
for i, stop in enumerate(route):
    day_info = "Day " +  str(stop[1].min_arrival_minutes // (24*60) + 1)
    print(f"{str(i+1).rjust(len(str(len(route))))}. \t{str(stop[1].last_route.line).rjust(max_length[0])}) [{format_time(stop[1].last_route.departure_minutes).rjust(max_length[2])}] {stop[1].last_stop.name.ljust(max_length[1])} - "
          f"[{format_time(stop[1].last_route.arrival_minutes)}] {stop[0].name} ({day_info})")

 1. 	  4) [23:54] Tramwajowa                        - [23:56] ZOO (Day 1)
 2. 	  4) [23:56] ZOO                               - [23:57] Hala Stulecia (Day 1)
 3. 	  4) [23:57] Hala Stulecia                     - [23:59] Kliniki - Politechnika Wrocławska (Day 1)
 4. 	  4) [23:59] Kliniki - Politechnika Wrocławska - [00:01] PL. GRUNWALDZKI (Day 2)
 5. 	  4) [00:01] PL. GRUNWALDZKI                   - [00:03] Piastowska (Day 2)
 6. 	  4) [00:03] Piastowska                        - [00:05] Prusa (Day 2)
 7. 	  4) [00:05] Prusa                             - [00:06] Wyszyńskiego (Day 2)
 8. 	  4) [00:06] Wyszyńskiego                      - [00:08] Nowowiejska (Day 2)
 9. 	  4) [00:08] Nowowiejska                       - [00:10] Słowiańska (Day 2)
10. 	250) [00:21] Słowiańska                        - [00:22] DWORZEC NADODRZE (Day 2)
11. 	250) [00:22] DWORZEC NADODRZE                  - [00:23] Paulińska (Day 2)
12. 	250) [00:23] Paulińska                         - [00:25] Dubois (Day 2)
13. 	

In [136]:
for k,v in stops_records.items():
    print(k,v.min_arrival_minutes)

Chełmońskiego[51.10388356, 17.09071195] 10000000000.0
Tramwajowa[51.10446678, 17.08466997] 1433
ZOO[51.10556081, 17.0778982] 2635
Hala Stulecia[51.10708825, 17.07346452] 4076
Kliniki - Politechnika Wrocławska[51.10971261, 17.06511141] 5518
PL. GRUNWALDZKI[51.11144435, 17.05995961] 6960
most Grunwaldzki[51.11021216, 17.05538488] 8401
Piastowska[51.11536582, 17.06087806] 8430
Prusa[51.11947637, 17.05740455] 9872
Wyszyńskiego[51.12191852, 17.05294165] 11313
Nowowiejska[51.12423914, 17.0448911] 12755
Słowiańska[51.12354187, 17.04021724] 14196
DWORZEC NADODRZE[51.12431442, 17.03503321] 15638


In [137]:
print(real_timetable['arrival_time'].agg(['min', 'max']))

min    03:36:00
max    30:05:00
Name: arrival_time, dtype: object


In [ ]:
print("Values range for departure time: " + real_timetable['departure_time'].min() + " - " + real_timetable['departure_time'].max())
print("Values range for arrival time: " + real_timetable['arrival_time'].min() + " - " + real_timetable['arrival_time'].max())
print("Values range for start-stop latitude: " + str(real_timetable['start_stop_lat'].min()) + " - " + str(real_timetable['start_stop_lat'].max()))
print("Values range for start-stop longitude: " + str(real_timetable['start_stop_lon'].min()) + " - " + str(real_timetable['start_stop_lon'].max()))

Przedzial departure time: 03:34:00 - 30:03:00
Przedzial arrival time: 03:36:00 - 30:05:00
Przedzial start-stop latitude: 50.98042601 - 51.26556584
Przedzial start-stop longitude: 16.695853 - 17.2855453
